In [3]:
!apt-get install swi-prolog -y
!pip install pyswip langchain-google-genai langgraph langchain-core

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-sendmail-perl libossp-uuid16 libsub-override-perl
  libsys-hostname-long-perl libtool po-debconf swi-prolog-core
  swi-prolog-core-packages swi-prolog-doc swi-prolog-nox swi-prolog-x
Suggested packages:
  dh-make gettext-doc libasprintf-dev libgettextpo-dev uuid libtool-doc
  gcj-jdk libmail-box-perl elpa-ediprolog swi-prolog-java swi-prolog-odbc
  swi-prolog-bdb
The following NEW packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-send

In [ ]:
import os
from pyswip import Prolog
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
import json

GOOGLE_API_KEY = "Use Your Own API Key Here"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [ ]:
class AdvancedPrologInterface:
    def __init__(self):
        self.prolog = Prolog()
        self._load_knowledge_base()

    def _load_knowledge_base(self):
        """Load comprehensive Prolog knowledge base"""
        rules = [
            "parent(john, mary, alice)",
            "parent(john, mary, bob)",
            "parent(bob, susan, charlie)",
            "parent(alice, david, emma)",
            "parent(charlie, lisa, frank)",

            "male(john)", "male(bob)", "male(david)", "male(charlie)", "male(frank)",
            "female(mary)", "female(alice)", "female(susan)", "female(emma)", "female(lisa)",

            "grandparent(X, Z) :- parent(X, _, Y), parent(Y, _, Z)",
            "sibling(X, Y) :- parent(P1, P2, X), parent(P1, P2, Y), X \\= Y",
            "uncle(X, Y) :- sibling(X, Z), parent(Z, _, Y), male(X)",
            "aunt(X, Y) :- sibling(X, Z), parent(Z, _, Y), female(X)",
            "cousin(X, Y) :- parent(P1, _, X), parent(P2, _, Y), sibling(P1, P2)",

            "factorial(0, 1)",
            "factorial(N, F) :- N > 0, N1 is N - 1, factorial(N1, F1), F is N * F1",

            "list_member(X, [X|_])",
            "list_member(X, [_|T]) :- list_member(X, T)",
            "list_length([], 0)",
            "list_length([_|T], N) :- list_length(T, N1), N is N1 + 1",

            "animal(dog)", "animal(cat)", "animal(whale)", "animal(eagle)",
            "mammal(dog)", "mammal(cat)", "mammal(whale)",
            "bird(eagle)", "bird(sparrow)",
            "can_fly(eagle)", "can_fly(sparrow)",
            "can_swim(whale)", "can_swim(fish)",
            "aquatic_mammal(X) :- mammal(X), can_swim(X)"
        ]

        for rule in rules:
            try:
                self.prolog.assertz(rule)
            except Exception as e:
                print(f"Warning: Could not assert rule '{rule}': {e}")

    def query(self, query_string):
        """Execute Prolog query and return results"""
        try:
            results = list(self.prolog.query(query_string))
            return results if results else [{"result": "No solutions found"}]
        except Exception as e:
            return [{"error": f"Query failed: {str(e)}"}]

In [ ]:
prolog_interface = AdvancedPrologInterface()

@tool
def family_relationships(query: str) -> str:
    """
    Query family relationships in Prolog format.
    Examples: 'parent(john, mary, X)', 'sibling(X, Y)', 'grandparent(X, charlie)'
    """
    results = prolog_interface.query(query)
    return json.dumps(results, indent=2)

@tool
def mathematical_operations(operation: str, number: int) -> str:
    """
    Perform mathematical operations using Prolog.
    Supported operations: 'factorial'
    Example: operation='factorial', number=5
    """
    if operation == "factorial":
        query = f"factorial({number}, Result)"
        results = prolog_interface.query(query)
        return json.dumps(results, indent=2)
    else:
        return json.dumps([{"error": f"Operation '{operation}' not supported"}])

@tool
def advanced_queries(query_type: str, entity: str = "") -> str:
    """
    Perform advanced relationship queries.
    Types: 'all_children', 'all_grandchildren', 'all_siblings', 'all_cousins'
    """
    queries = {
        'all_children': f"parent(_, _, {entity})" if entity else "parent(_, _, X)",
        'all_grandchildren': f"grandparent(_, {entity})" if entity else "grandparent(_, X)",
        'all_siblings': f"sibling({entity}, X)" if entity else "sibling(X, Y)",
        'all_cousins': f"cousin({entity}, X)" if entity else "cousin(X, Y)"
    }

    if query_type in queries:
        results = prolog_interface.query(queries[query_type])
        return json.dumps(results, indent=2)
    else:
        return json.dumps([{"error": f"Query type '{query_type}' not supported"}])

In [ ]:
tools = [family_relationships, mathematical_operations, advanced_queries]
agent = create_react_agent(llm, tools)

def run_family_analysis():
    """Comprehensive family relationship analysis"""
    print("👨‍👩‍👧‍👦 Family Relationship Analysis")
    print("=" * 50)

    queries = [
        "Who are all the parents in the family database?",
        "Find all grandparent-grandchild relationships",
        "Show me all the siblings in the family",
        "Who are John and Mary's children?",
        "Calculate the factorial of 6 using Prolog"
    ]

    for i, query in enumerate(queries, 1):
        print(f"\n🔍 Query {i}: {query}")
        print("-" * 30)

        try:
            response = agent.invoke({"messages": [("human", query)]})
            answer = response["messages"][-1].content
            print(f"🤖 Response: {answer}")
        except Exception as e:
            print(f"❌ Error: {str(e)}")

def demonstrate_complex_reasoning():
    """Show advanced multi-step reasoning"""
    print("\n🧠 Complex Multi-Step Reasoning")
    print("=" * 40)

    complex_query = """
    I want a complete family tree analysis. Please:
    1. List all parent-child relationships
    2. Identify all grandparent relationships
    3. Find any uncle/aunt relationships
    4. Show cousin relationships
    5. Calculate factorial of 4 as a bonus math operation
    """

    print(f"Complex Query: {complex_query}")
    print("-" * 40)

    try:
        response = agent.invoke({"messages": [("human", complex_query)]})
        print(f"📋 Comprehensive Analysis:\n{response['messages'][-1].content}")
    except Exception as e:
        print(f"❌ Error in complex reasoning: {str(e)}")

def interactive_prolog_session():
    """Interactive Prolog knowledge base exploration"""
    print("\n💬 Interactive Prolog Explorer")
    print("Ask about family relationships, math operations, or general queries!")
    print("Type 'examples' to see sample queries, 'quit' to exit")
    print("-" * 50)

    examples = [
        "Who are Bob's children?",
        "Find all grandparents in the family",
        "Calculate factorial of 5",
        "Show me all cousin relationships",
        "Who are Alice's siblings?"
    ]

    while True:
        user_input = input("\n🧑 You: ")

        if user_input.lower() == 'quit':
            print("👋 Goodbye!")
            break
        elif user_input.lower() == 'examples':
            print("📝 Example queries:")
            for ex in examples:
                print(f"  • {ex}")
            continue

        try:
            response = agent.invoke({"messages": [("human", user_input)]})
            print(f"🤖 AI: {response['messages'][-1].content}")
        except Exception as e:
            print(f"❌ Error: {str(e)}")

In [1]:
def test_direct_queries():
    """Test direct Prolog queries for verification"""
    print("\n🔬 Direct Prolog Query Testing")
    print("=" * 35)

    test_queries = [
        ("parent(john, mary, X)", "Find John and Mary's children"),
        ("grandparent(X, charlie)", "Find Charlie's grandparents"),
        ("sibling(alice, X)", "Find Alice's siblings"),
        ("factorial(4, X)", "Calculate 4 factorial"),
        ("cousin(X, Y)", "Find all cousin pairs")
    ]

    for query, description in test_queries:
        print(f"\n📋 {description}")
        print(f"Query: {query}")
        results = prolog_interface.query(query)
        print(f"Results: {json.dumps(results, indent=2)}")


def main():
    """Main demonstration runner"""
    if GOOGLE_API_KEY == "YOUR_GEMINI_API_KEY_HERE":
        print("⚠️  Please set your Gemini API key in Cell 3!")
        print("Get it from: https://aistudio.google.com/app/apikey")
        return

    print("🚀 Advanced Prolog + Gemini Integration")
    print("Using PySwip for stable Prolog integration")
    print("=" * 55)

    test_direct_queries()
    run_family_analysis()
    demonstrate_complex_reasoning()


def show_mathematical_capabilities():
    """Demonstrate mathematical reasoning with Prolog"""
    print("\n🔢 Mathematical Reasoning with Prolog")
    print("=" * 40)

    math_queries = [
        "Calculate factorial of 3, 4, and 5",
        "What is the factorial of 7?",
        "Show me how factorial calculation works step by step"
    ]

    for query in math_queries:
        print(f"\n🧮 Math Query: {query}")
        try:
            response = agent.invoke({"messages": [("human", query)]})
            print(f"📊 Result: {response['messages'][-1].content}")
        except Exception as e:
            print(f"❌ Error: {str(e)}")

if __name__ == "__main__":
    main()
    show_mathematical_capabilities()

    print("\n✅ Tutorial completed successfully!")
    print("🎯 Key achievements:")
    print("  • Integrated PySwip with Gemini AI")
    print("  • Created advanced Prolog reasoning tools")
    print("  • Demonstrated complex family relationship queries")
    print("  • Implemented mathematical operations in Prolog")
    print("  • Built interactive AI agent with logical reasoning")
    print("\n🚀 Try extending with your own Prolog rules and facts!")

🚀 Advanced Prolog + Gemini Integration
Using PySwip for stable Prolog integration

🔬 Direct Prolog Query Testing

📋 Find John and Mary's children
Query: parent(john, mary, X)
Results: [
  {
    "X": "alice"
  },
  {
    "X": "bob"
  }
]

📋 Find Charlie's grandparents
Query: grandparent(X, charlie)
Results: [
  {
    "X": "john"
  }
]

📋 Find Alice's siblings
Query: sibling(alice, X)
Results: [
  {
    "X": "bob"
  }
]

📋 Calculate 4 factorial
Query: factorial(4, X)
Results: [
  {
    "X": 24
  }
]

📋 Find all cousin pairs
Query: cousin(X, Y)
Results: [
  {
    "X": "charlie",
    "Y": "emma"
  },
  {
    "X": "emma",
    "Y": "charlie"
  }
]
👨‍👩‍👧‍👦 Family Relationship Analysis

🔍 Query 1: Who are all the parents in the family database?
------------------------------
🤖 Response: The available tools don't directly list all parents.  To get that information, I need to query the database for all individuals who have children.  However, the `family_relationships` function requires a specif